In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
import pickle

def load_processed_data(file_path):
    with open(file_path, 'rb') as file:
        X, Y = pickle.load(file)
    return X, Y

In [10]:
X, Y = load_processed_data('./pickel_files/res_processed_data_buffer.pkl')

In [11]:

X_filtered = [img for img in X if img is not None and isinstance(img, np.ndarray)]
Y_filtered = [Y[i] for i in range(len(Y)) if X[i] is not None and isinstance(X[i], np.ndarray)]

X_filtered = np.array(X_filtered)

Y_filtered = np.array(Y_filtered)
Y_filtered = Y_filtered[:, :2]

In [12]:
len(X_filtered), len(Y_filtered)

(3948, 3948)

In [13]:
Y_filtered.shape

(3948, 2)

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X_filtered, Y_filtered, test_size=0.2, random_state=42)
#Val data
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

train_datagen = ImageDataGenerator(
    # rotation_range=5,
    # width_shift_range=0.02,
    # height_shift_range=0.02,
    # zoom_range=[0.95, 1.05],
    # brightness_range=[0.8, 1.2],
    # shear_range=0.1,
    # fill_mode='nearest'
)


# Assuming you have your training data in train_data and train_labels
train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)


model = Sequential()

# Adding L2 Regularization to Convolutional Layers
l2_reg = 0.001

# First Conv Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 80, 3), kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D())
model.add(Dropout(0.1))

# Second Conv Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.15))

# Third Conv Block
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D())
model.add(BatchNormalization())


# Fourth Conv Block
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))

# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00005), loss='mse', metrics=['mean_squared_error', 'mean_absolute_error'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=100,  # Adjust number of epochs
    validation_data=(X_val, Y_val),  # Assuming validation data is available
      callbacks=[early_stopping],
    batch_size=16
)

Epoch 1/100
79/79 [==============================] - 7s 77ms/step - loss: 0.1833 - mean_squared_error: 0.0939 - mean_absolute_error: 0.2530 - val_loss: 0.1936 - val_mean_squared_error: 0.1055 - val_mean_absolute_error: 0.2812
Epoch 2/100
79/79 [==============================] - 6s 75ms/step - loss: 0.1580 - mean_squared_error: 0.0712 - mean_absolute_error: 0.2132 - val_loss: 0.1873 - val_mean_squared_error: 0.1019 - val_mean_absolute_error: 0.2761
Epoch 3/100
79/79 [==============================] - 6s 74ms/step - loss: 0.1476 - mean_squared_error: 0.0635 - mean_absolute_error: 0.1984 - val_loss: 0.1802 - val_mean_squared_error: 0.0974 - val_mean_absolute_error: 0.2696
Epoch 4/100
79/79 [==============================] - 6s 72ms/step - loss: 0.1402 - mean_squared_error: 0.0587 - mean_absolute_error: 0.1894 - val_loss: 0.1712 - val_mean_squared_error: 0.0909 - val_mean_absolute_error: 0.2598
Epoch 5/100
79/79 [==============================] - 8s 100ms/step - loss: 0.1350 - mean_squared

In [16]:
#evaluate the model
model.evaluate(X_test, Y_test)

25/25 [==============================] - 1s 21ms/step - loss: 0.0317 - mean_squared_error: 0.0205 - mean_absolute_error: 0.0998


[0.031721487641334534, 0.02045309729874134, 0.0997592881321907]

In [ ]:
from keras.models import load_model
model = load_model('./models/eye_gaze_v13.h5')

In [17]:
#plot predicted vs actual on test data on a canvas using opencv 

import cv2
import numpy as np 
predictions = model.predict(X_test)
screen_width, screen_height = 2650, 1440
canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
cv2.namedWindow('Gaze Tracking on Canvas', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Gaze Tracking on Canvas', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# plot the first 10 images one by one
for i in range(0,2):
    
    # get the predicted x,y coordinates
    x, y = predictions[i][0] * screen_width, predictions[i][1] * screen_height

    # lock the preds 
    x = min(max(x, 0), screen_width)
    y = min(max(y, 0), screen_height)

    # get the actual x,y coordinates
    x_actual, y_actual = Y_test[i][0] * screen_width, Y_test[i][1] * screen_height

    # plot the predicted x,y coordinates
    cv2.circle(canvas, (int(x), int(y)), 10, (0, 0, 255), -1)
    # plot the actual x,y coordinates
    cv2.circle(canvas, (int(x_actual), int(y_actual)), 10, (0, 255, 0), -1 )
    # show the canvas
    cv2.imshow('Gaze Tracking on Canvas', canvas)
    cv2.waitKey(0)
    # # show the image 
    # cv2.imshow('image', X_test[i])
    # cv2.waitKey(0)
    # # clear the canvas
    
    canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)

cv2.waitKey(0)
cv2.destroyAllWindows()

25/25 [==============================] - 1s 22ms/step


In [18]:
model.save('./models/eye_gaze_v15_sigmoidv7.h5')